In [53]:
import pandas as pd
import numpy as np
data = pd.read_csv("../data/enavi201605(6271).csv",encoding="shift_jis")

In [98]:
#店舗ごとの総額
pd.DataFrame(data.groupby("利用店名・商品名")["利用金額"].sum()).head(5)

,利用金額
利用店名・商品名,
AMAZON DOWNLOADS,19268
AUDIBLE / AMZN,1500
BASE*ﾅﾐﾘｰ,1960
BILL ITUNES COM,2080
DMM,2198


In [100]:
#店舗ごとの総額　ピボットテーブル
price = pd.DataFrame(pd.pivot_table(data,values='支払総額',index=['利用店名・商品名'],aggfunc=np.sum))
price.head(5)

,支払総額
利用店名・商品名,
AMAZON DOWNLOADS,19268
AUDIBLE / AMZN,1500
BASE*ﾅﾐﾘｰ,1960
BILL ITUNES COM,2080
DMM,2198


In [111]:
#店舗ごとの利用回数
group2 = data.groupby(['利用店名・商品名','利用日'])
group2.count().head(5)

新規サイン  利用者  支払方法  利用金額  支払手数料  支払総額  支払回数/何回目  \
利用店名・商品名         利用日                                                       
AMAZON DOWNLOADS 16.04.06      1    1     1     1      1     1         1   
                 16.04.08      2    2     2     2      2     2         2   
                 16.04.09      2    2     2     2      2     2         2   
                 16.04.14      1    1     1     1      1     1         1   
                 16.04.16      3    3     3     3      3     3         3   

                           当月請求額  翌月繰越残高  
利用店名・商品名         利用日                      
AMAZON DOWNLOADS 16.04.06      1       1  
                 16.04.08      2       2  
                 16.04.09      2       2  
                 16.04.14      1       1  
                 16.04.16      3       3

In [110]:
#カラム別に計算方法を変える
group1 = data.groupby(['利用店名・商品名'])['利用金額']
group1.agg([('利用金額',np.mean),('支払総額',np.sum),('利用店名・商品名','count')]).head(5)

,利用金額,支払総額,利用店名・商品名
利用店名・商品名,,,
AMAZON DOWNLOADS,535.222222,19268,36
AUDIBLE / AMZN,1500.000000,1500,1
BASE*ﾅﾐﾘｰ,1960.000000,1960,1
BILL ITUNES COM,1040.000000,2080,2
DMM,2198.000000,2198,1


In [113]:
#カラムごとに複数計算を同時に行う
functions = ['count' , 'mean' , 'sum']
group2['利用金額','支払総額'].agg(functions).head(5)

利用金額                    支払総額                  
                          count        mean   sum count        mean   sum
利用店名・商品名         利用日                                                     
AMAZON DOWNLOADS 16.04.06     1  400.000000   400     1  400.000000   400
                 16.04.08     2  896.000000  1792     2  896.000000  1792
                 16.04.09     2  432.000000   864     2  432.000000   864
                 16.04.14     1  432.000000   432     1  432.000000   432
                 16.04.16     3  572.666667  1718     3  572.666667  1718

In [114]:
#カラムごとに別の計算をする
group2.agg({'利用金額' : ['count', 'sum'], '支払総額' :['mean']}).head(5)

利用金額              支払総額
                          count   sum        mean
利用店名・商品名         利用日                             
AMAZON DOWNLOADS 16.04.06     1   400  400.000000
                 16.04.08     2  1792  896.000000
                 16.04.09     2   864  432.000000
                 16.04.14     1   432  432.000000
                 16.04.16     3  1718  572.666667

In [95]:
#利用店ごとの週ごとの合計金額
data2 = data

data2.head(10)

print(len(data2))
#不要な行を削除
data2 = data2.dropna(subset=['利用日'])

#利用日が文字列なので日付に変換
data2['利用日'] = '20' + data2['利用日']
data2['利用日'] = pd.to_datetime(data2['利用日'])

#利用店と日付の合計金額をクロス集計
daily_price = data2.groupby(['利用店名・商品名' , '利用日'])['利用金額'].sum()

#週ごとに数値をまとめる
daily_price = daily_price.unstack().fillna(0)
daily_price = daily_price.resample('W',how='sum' ,axis=1)
daily_price.T

/Users/tetsubones/.pyenv/versions/anaconda3-2.4.0/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tetsubones/.pyenv/versions/anaconda3-2.4.0/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


115


/Users/tetsubones/.pyenv/versions/anaconda3-2.4.0/lib/python3.5/site-packages/pandas/core/index.py:4281: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


利用店名・商品名,AMAZON DOWNLOADS,AUDIBLE / AMZN,BASE*ﾅﾐﾘｰ,BILL ITUNES COM,DMM,GOOGLE *DENA利用国GB,セブン−イレブン,レストラン オマージュ,ローソン,＠ｎｉｆｔｙ,...,ｼﾙｺ,ﾃﾞｲ-ｴﾑｴﾑ,ﾃﾞｲﾘ-ﾔﾏｻﾞｷ,ﾄﾞﾗﾂｸﾞｾｲﾑｽｽﾐﾀﾞﾘﾖｳ,ﾄﾞﾗﾂｸﾞｾｶﾞﾐｵｶﾁﾏﾁｴｷ,ﾈｽﾚﾈｽﾌﾟﾚﾂｿ,ﾌｱﾐﾘ-ﾏ-ﾄﾆｼｼﾝｼﾞﾕｸﾊ,ﾏﾙｺﾞﾄﾆﾂﾎﾟﾝ,ﾐﾆｽﾄﾂﾌﾟｼﾅﾉﾏﾁｴｷﾐﾅﾐ,ﾓﾄﾏﾁﾕﾆｵﾝ
利用日,,,,,,,,,,,,,,,,,,,,,
2016-04-03,0,0,1960,0,0,0,2358,0,108,974,...,0,0,1198,0,0,0,0,0,0,0
2016-04-10,3056,0,0,1480,0,0,1466,0,0,0,...,0,3620,0,0,1514,5634,0,0,0,511
2016-04-17,2150,1500,0,600,0,0,2338,0,1206,0,...,0,7016,0,0,0,0,0,0,0,0
2016-04-24,4666,0,0,0,0,900,2735,0,1835,0,...,9650,1406,0,0,0,0,460,1020,0,0
2016-05-01,9396,0,0,0,2198,0,158,45500,438,0,...,0,2256,0,2187,0,0,0,0,1355,1581
